In [ ]:
!pip install -r "requirements.txt"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-pn8sirpe
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-pn8sirpe
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.5 MB/s eta 0:00:00
     ━━

In [ ]:
!pip install wandb
!wandb login b0dbcaaefb7898cfcce83b4db38fb742e9063b13

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=e5139f19971609f9c966d42c8204577a5885cd970204dd9032bf71385da60a79
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:

import gc
import io


import random
import numpy as np

import torchvision
import transformers
import more_itertools
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset
from tqdm import tqdm
from dataclasses import dataclass, field
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2
from PIL import Image
import pickle
import sys
from tqdm.contrib import tzip
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as nnf
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup

import os
import pickle
import sys
import argparse
import json
from typing import Tuple, Optional, Union
from torch.cuda.amp import autocast

from transformers.optimization import Adafactor, AdafactorSchedule
import wandb
import torch

from torch.utils.checkpoint import checkpoint_sequential

class ClipCocoDataset(Dataset):
    
    def __init__(self, data_path: str,  prefix_length= 50, gpt2_type = "sberbank-ai/rugpt3small_based_on_gpt2",
                 normalize_prefix=True):
        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.prefix_length = prefix_length
        self.normalize_prefix = normalize_prefix
        with open(data_path, 'rb') as f:
            all_data = pickle.load(f)
        print("Data size is %0d" % len(all_data["clip_embedding"]))
        sys.stdout.flush()
        self.prefixes = all_data["clip_embedding"]
        captions_raw = all_data["captions"]
        
        #self.image_ids = [caption["image_id"] for caption in captions_raw]
        
        self.captions = captions_raw
        
        
        self.captions_tokens = []
        self.caption2embedding = []
        max_seq_len = 0
        i=0
        for caption in tqdm(captions_raw):
                self.captions_tokens.append(torch.tensor(self.tokenizer.encode(caption), dtype=torch.int64))
                self.caption2embedding.append(self.prefixes[i])
                i+=1
                max_seq_len = max(max_seq_len, self.captions_tokens[-1].shape[0])
            # self.max_seq_len = max_seq_len
        #del self.captions_tokens
        #del self.caption2embedding
        #gc.collect()
        #with open(f"{data_path[:-4]}_tokens.pkl", 'wb') as f:
        #        pickle.dump([self.captions_tokens, self.caption2embedding, max_seq_len], f)
       
    
    
        all_len = torch.tensor([len(self.captions_tokens[i]) for i in range(len(self))]).float()
        self.max_seq_len = min(int(all_len.mean() + all_len.std() * 10), int(all_len.max()))

    def pad_tokens(self, item: int):
        tokens = self.captions_tokens[item]
        padding = self.max_seq_len - tokens.shape[0]
        if padding > 0:
            tokens = torch.cat((tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            self.captions_tokens[item] = tokens
        elif padding < 0:
            tokens = tokens[:self.max_seq_len]
            self.captions_tokens[item] = tokens
        mask = tokens.ge(0)  # mask is zero where we out of sequence
        tokens[~mask] = 0
        mask = mask.float()
        mask = torch.cat((torch.ones(self.prefix_length), mask), dim=0)  # adding prefix mask
        return tokens, mask
    
    def __len__(self) -> int:
        return len(self.captions_tokens)

   

    def __getitem__(self, item):
        tokens, mask = self.pad_tokens(item)
        prefix = self.prefixes[item]
        if self.normalize_prefix:
            prefix = prefix.float()
            prefix = prefix / prefix.norm(2, -1)
        return tokens, mask, prefix

    

In [ ]:
class MLP(nn.Module):
    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            layers.append(nn.Dropout(0.2))
            layers.append(nn.ReLU())
        self.model = nn.Sequential(*layers)
    
    #@autocast()  
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    
def freeze(
    model,
    freeze_emb=False,
    freeze_ln=False,
    freeze_attn=True,
    freeze_ff=False,
    freeze_other=True,
):
    
    for name, p in model.named_parameters():       
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
           
    return model

class ClipCaptionModel(nn.Module):
    def __init__(self, backbone, prefix_length: int, prefix_size: int = 768):
          super(ClipCaptionModel, self).__init__()
          self.prefix_length = prefix_length
          self.gpt = GPT2LMHeadModel.from_pretrained(backbone)
          self.gpt = freeze(self.gpt)
          self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
          self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2,
                                  self.gpt_embedding_size * prefix_length))

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)
    
    # @autocast() 
    def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None):

        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)

        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

  


class ClipCaptionPrefix(ClipCaptionModel):

    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self

In [ ]:
import os
wandb.init(project="clip_caption_video")

def train(train_dataset, valid_dataset, model: ClipCaptionModel, args,
          warmup_steps: int = 1000, output_dir: str = ".", output_prefix: str = ""):

  device = torch.device('cuda')# xm.xla_device()
  #
  batch_size = args.bs
  epochs = args.epochs
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  
  model = model.to(device)
  
  model = freeze(model)
  optimizer = AdamW(model.parameters(), lr=args.lr,betas=(0.9, 0.995))
  #optimizer = bnb.optim.Adam8bit(model.parameters(), lr=0.001, betas=(0.9, 0.995))
  #optimizer = SM3(model.parameters(),lr=args.lr)
  #Adafactor(model.parameters(),scale_parameter=True, relative_step=True, warmup_init=True, lr=None)

  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
  scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs * len(train_dataloader)
  )

  valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
  #AdafactorSchedule(optimizer)#num_training_steps=epochs * len(train_dataloader
  #save_config(args)
  #print

  for epoch in range(epochs):
    print(f">>> Training epoch {epoch}")
    sys.stdout.flush()
    step=0
    running_loss = 0
    model.train()
    with tqdm(total=len(train_dataloader), desc=output_prefix) as progress_train:
      for idx, (tokens, mask, prefix) in enumerate(train_dataloader):
        model.zero_grad()
        step+=1
        tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)
        
        outputs = model(tokens, prefix, mask)
        logits = outputs.logits[:, dataset_train.prefix_length - 1: -1]

        loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)

        segments = 2

        
        #out = checkpoint_sequential(modules, segments, input_var)

        # backpropagate
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        #optimizer.zero_grad()
        progress_train.set_postfix({"loss": loss.item()})
        running_loss += loss.item()
        
        clipping_value = 0.5 # arbitrary value of your choosing
        #torch.nn.utils.clip_grad_norm(model.parameters(), clipping_value)
        optimizer.step()
        scheduler.step()
        
        wandb.log({"loss":  loss.item()})
        
        progress_train.update()
        

        del tokens
        del mask
        del prefix
        torch.clear_autocast_cache()
        torch.cuda.empty_cache()
        
        if (idx + 1) % 1000 == 0:
          torch.save(
            model.state_dict(),
            os.path.join(output_dir, f"{output_prefix}_latest.pt"),
          )
          
    print(f'Loss: {running_loss/len(train_dataloader)}')
    #validation
    
    step=0
    running_loss = 0
    model.eval()
    with tqdm(total=len(valid_dataloader), desc=output_prefix) as progress_valid:
      with torch.no_grad():
        for idx, (tokens, mask, prefix) in enumerate(valid_dataloader):
          step+=1
          tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)
          
          outputs = model(tokens, prefix, mask)
          logits = outputs.logits[:, dataset_valid.prefix_length - 1: -1]

          loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)

          progress_valid.set_postfix({"valid_loss": loss.item()})
          wandb.log({"valid_loss":  loss.item()})
          running_loss += loss.item()

          progress_valid.update()

          torch.clear_autocast_cache()
          torch.cuda.empty_cache()
    print(f'Validation Loss: {running_loss/len(valid_dataloader)}')

    if epoch % args.save_every ==0:
      torch.save(
        model.state_dict(),
        os.path.join(output_dir, f"{output_prefix}-{epoch:03d}.pt"),
      )
      os.system(f'cp /content/checkpoints_larger/{output_prefix}-{epoch:03d}.pt /content/drive/MyDrive/checkpoints')
  return model



class Args():
  def __init__(self):
    self.backbone = 'sberbank-ai/rugpt3small_based_on_gpt2'
    self.data_train = 'Features_train_full_ru.pkl'
    self.data_valid = 'Features_valid_full_ru.pkl'
    self.out_dir = 'checkpoints_larger'
    self.prefix = 'prefix_1'
    self.epochs = 10
    self.save_every = 1
    self.prefix_length = 50
    self.bs = 10
    self.only_prefix = False
    self.lr = 5e-4
  
    

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alexixplay. Use `wandb login --relogin` to force relogin


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!rm -r checkpoints*

rm: cannot remove 'checkpoints*': No such file or directory


In [ ]:
args = Args()
wandb.config = {
  "learning_rate": args.lr,
  "epochs": args.epochs,
  "batch_size": args.bs
}

prefix_length = args.prefix_length

dataset_train = ClipCocoDataset(args.data_train, prefix_length)
dataset_valid = ClipCocoDataset(args.data_valid, prefix_length)


#model_path = 'prefix_1-003.pt'
model = ClipCaptionModel(backbone = args.backbone, prefix_length = 50)
# model.load_state_dict(torch.load(model_path, map_location='cpu')) 
#print("Train both prefix and GPT")
sys.stdout.flush()
train(dataset_train, dataset_valid, model, args, output_dir=args.out_dir, output_prefix=args.prefix)

Data size is 16794


100%|██████████| 16794/16794 [00:07<00:00, 2362.17it/s]


Data size is 4199


100%|██████████| 4199/4199 [00:03<00:00, 1326.05it/s]


>>> Training epoch 0


...

Loss: 2.090956102388824


prefix_1: 100%|██████████| 419/419 [00:39<00:00, 10.71it/s, valid_loss=2.02]


Validation Loss: 1.8914244695608826
>>> Training epoch 1


prefix_1:   2%|▏         | 26/1679 [00:07<08:15,  3.34it/s, loss=1.68]


KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

In [ ]:
!cp /content/checkpoints_larger/prefix_1-000.pt /content/drive/MyDrive/checkpoints